In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using RigidBodyTreeInspector
import LCPSim
using JuMP
using Gurobi
import ConditionalJuMP
import LearningMPC

In [ ]:
reload("LCPSim")
reload("LearningMPC")

In [ ]:
urdf_mech = parse_urdf(Float64, "box_robot.urdf")
mechanism, base = LCPSim.planar_revolute_base()
attach!(mechanism, base, urdf_mech)
world = root_body(mechanism)
foot = findbody(mechanism, "lf")
env = LCPSim.Environment{Float64}(Dict())
xstar = MechanismState{Float64}(mechanism)
set_configuration!(xstar, findjoint(mechanism, "base_z"), [0.8])
set_configuration!(xstar, findjoint(mechanism, "core_to_lf_extension"), [0.6])
params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=2,
    mip_solver=GurobiSolver(Gurobi.Env(), TimeLimit=120, MIPGap=5e-2),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

In [ ]:
vis = Visualizer()[:box_robot]
setgeometry!(vis, mechanism, parse_urdf("box_robot.urdf", mechanism))
settransform!(vis, xstar)

In [ ]:
Q = diagm([10, 100, 1, 0.1, 0.1, 1, 1, 0.1, 0.1, 0.1])
R = diagm(fill(0.01, 5))
contacts = [Point3D(default_frame(findbody(mechanism, "lf")), 0., 0., 0.)]
qstar = copy(configuration(xstar))
vstar = copy(velocity(xstar))
lqr = LearningMPC.LQRSolution(xstar, Q, R, contacts, params.Δt)

In [ ]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_z"), [5.0])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))

In [ ]:
results = LCPSim.simulate(x0, lqr, env, params.Δt, 100, params.lcp_solver);

In [ ]:
LearningMPC.playback(vis, results, params.Δt)

In [ ]:
last_trajectory = []
mpc_controller = LearningMPC.MPCController(mechanism, env, params, lqr, [lqr])
mpc_controller.callback = (x, mpcresults) -> begin
    global last_trajectory
    if !isnull(mpcresults.lcp_updates)
        last_trajectory = get(mpcresults.lcp_updates)
        LearningMPC.playback(vis, get(mpcresults.lcp_updates), params.Δt)
    end
end

In [ ]:
x0 = MechanismState(mechanism, copy(configuration(xstar)), copy(velocity(xstar)))
set_velocity!(x0, findjoint(mechanism, "base_x"), [0.5])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
mpc_controller(x0)

In [ ]:
LearningMPC.playback(vis, last_trajectory, params.Δt)